# CoreScale results for "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [3]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [4]:
slice = fablib.get_slice(name=slice_name)

In [5]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [6]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Configure router for CoreScale scenario

> CoreScale: The “at scale" setting with a bottleneck bandwidth
of 10 Gbps, 1000 to 5000 competing flows, and a 375MB
buffer

We implement CoreScale at the router using a `tc htb` rate limiting queue. Because of [limitations of the software implementation](https://unix.stackexchange.com/questions/704918/has-10-gbps-through-linux-tc-qdiscs-ever-been-solved), we use hardware offload on the Mellanox interface to achieve the full 10 Gbps.

First, make sure this feature is turned on:

In [7]:
router_node.execute('sudo ethtool  -K ' + router_egress_name + ' hw-tc-offload on')
router_node.execute('sudo ethtool  -k ' + router_egress_name + " | grep hw-tc-offload" )

hw-tc-offload: on


('hw-tc-offload: on\n', '')

Now, we should be able to configure the 10 Gbps queue.

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [8]:
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

 Error: Cannot delete qdisc with handle of zero.
 

('', 'Error: Cannot delete qdisc with handle of zero.\n')

Then set one up, with hardware offload:

In [9]:
router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3 offload")
router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate 10Gbit")
router_node.execute("sudo tc qdisc add dev " + router_egress_name + " parent 1:3 handle 3: bfifo limit 375MB")

('', '')

## Check network capacity

This time, we should get close to **10 Gbps** through the router.

Now start the `iperf3` flows:

In [10]:
for n in receiver_nodes:
    n.execute("iperf3 -s -1 -f g -D --logfile validate.dat")

In [11]:
import time
for i, n in enumerate(sender_nodes):
    n.execute_thread("iperf3 -t 60 -P 10 -c 10.10.2.1"+str(i))
time.sleep(65)

In [12]:
tputs = []
for n in receiver_nodes:
    (nout, nerr) = n.execute("tail --lines=2 validate.dat | grep receiver | awk '{print $6}'")
    tputs.append(float(nout.strip()))
print("Sum throughput is: %f Gbps" % sum(tputs))

0.89
0.87
0.93
1.07
1.05
0.78
1.02
1.01
1.07
1.03
Sum throughput is: 9.720000 Gbps


## Check network delay

We should still get sub-ms latency across the router.

In [13]:
for i, n in enumerate(sender_nodes):
    n.execute("ping -c 5 10.10.2.1"+str(i)+" | grep rtt")

rtt min/avg/max/mdev = 0.114/0.149/0.250/0.052 ms
rtt min/avg/max/mdev = 0.117/0.158/0.266/0.057 ms
rtt min/avg/max/mdev = 0.112/0.142/0.229/0.044 ms
rtt min/avg/max/mdev = 0.137/0.158/0.200/0.025 ms
rtt min/avg/max/mdev = 0.136/0.188/0.259/0.043 ms
rtt min/avg/max/mdev = 0.131/0.172/0.252/0.041 ms
rtt min/avg/max/mdev = 0.143/0.180/0.247/0.044 ms
rtt min/avg/max/mdev = 0.122/0.159/0.279/0.062 ms
rtt min/avg/max/mdev = 0.130/0.170/0.235/0.037 ms
rtt min/avg/max/mdev = 0.112/0.136/0.208/0.036 ms
